# Python Testing Demo

## Why testing is valuable in Python

## Different methods and packages for testing in Python

## Some tips & tricks

## Some gotchas

## Why testing is valuable in Python

- Proof that you delivered what was expected from you
- Prevent unexpected behavior
- Document both the functionality as well as the behavior of the code
- Make sure that what worked still works after code changes


In [1]:
def function_that_returns_text(text: str):
    if isinstance(text, str):
        return text
    else:
        raise ValueError(f"Input did not have a '{str}' type, but '{type(text)}'!")

print(function_that_returns_text("You should get this text back!"))

You should get this text back!


This code does exactly what it says it does, but you *proof* that it will by using tests to describe its behavior.

In [2]:
# Set up PyTest
!pip install ipytest
import ipytest
ipytest.autoconfig()

In [3]:
def test_str():
    assert function_that_returns_text("You should see this text!") == "You should see this text!"

def test_int_as_str():
    assert function_that_returns_text("15") == "15"

def test_int():
    assert function_that_returns_text(15) == "15"

In [4]:
ipytest.run('-vv')

======================================= test session starts ========================================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pyproject.toml
plugins: anyio-4.4.0
collecting ... collected 3 items

t_68541e827e4647818ea4c8724c7d6f58.py::test_str PASSED                                       [ 33%]
t_68541e827e4647818ea4c8724c7d6f58.py::test_int_as_str PASSED                                [ 66%]
t_68541e827e4647818ea4c8724c7d6f58.py::test_int FAILED                                       [100%]

============================================= FAILURES =============================================
_____________________________________________ test_int _____________________________________________

    def test_int():
>       assert function_that_returns_text(15) == "15"

/tmp/ipykernel_13273/1366318873.py:8: 
_ _ _ _ _ _ _ _ _ _ _ 

<ExitCode.TESTS_FAILED: 1>

### HTTP Responses

Sometimes you'll need to verify that certain HTTP requests contain specific data, i.e. to verify certain third-party behavior. You can catch and expose HTTP request data with the following code.

In [5]:

import logging
import pytest

@pytest.fixture
def debug_http(scope="module", autouse=False):
    """
    Catches all out-going HTTP calls and prints them in the console.
    Use `autouse=True` to automatically use it in every test which imports this conftest.py file.
    """
    import http.client as http_client

    http_client.HTTPConnection.debuglevel = 1
    logging.basicConfig()
    logging.getLogger("debug_http").setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

In [6]:
%%ipytest -s -vvv -k test_http_logs_are_not_catched
import requests

def test_http_logs_are_not_catched(
    capsys,
):
    requests.get("https://www.google.com/")

    out, err = capsys.readouterr()
    print(out)
    assert "reply: 'HTTP/1.1 200 OK\r\n'" not in out

ipytest.run('-vv')

======================================= test session starts ========================================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pyproject.toml
plugins: anyio-4.4.0
collecting ... collected 1 item

t_68541e827e4647818ea4c8724c7d6f58.py::test_http_logs_are_not_catched PASSED                 [100%]

======================================== 1 passed in 0.08s =========================================


<ExitCode.OK: 0>

======================================= test session starts ========================================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pyproject.toml
plugins: anyio-4.4.0
collecting ... collected 1 item

t_68541e827e4647818ea4c8724c7d6f58.py::test_http_logs_are_not_catched <- ../../tmp/ipykernel_13273/106178705.py 
PASSED

======================================== 1 passed in 0.07s =========================================


In [7]:
%%ipytest -s -vvv -k test_http_logs_are_catched

def test_http_logs_are_catched(
    debug_http,
    capsys,
):
    requests.get("https://www.google.com/")

    out, err = capsys.readouterr()
    print(out)
    assert "reply: 'HTTP/1.1 200 OK\r\n'" not in out
ipytest.run('-vv')

======================================= test session starts ========================================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pyproject.toml
plugins: anyio-4.4.0
collecting ... collected 1 item

t_68541e827e4647818ea4c8724c7d6f58.py::test_http_logs_are_catched PASSED                     [100%]

======================================== 1 passed in 0.09s =========================================


<ExitCode.OK: 0>

======================================= test session starts ========================================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pyproject.toml
plugins: anyio-4.4.0
collecting ... collected 1 item

t_68541e827e4647818ea4c8724c7d6f58.py::test_http_logs_are_catched <- ../../tmp/ipykernel_13273/1854283910.py send: b'GET / HTTP/1.1\r\nHost: www.google.com\r\nUser-Agent: python-requests/2.32.3\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Tue, 25 Jun 2024 12:36:11 GMT
header: Expires: -1
header: Cache-Control: private, max-age=0
header: Content-Type: text/html; charset=ISO-8859-1
header: Content-Security-Policy-Report-Only: object-src 'none';base-uri 'self';script-src 'nonce-proj3tZ9XwJ1ffFdrD75jg' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-